In [ ]:
import pandas as pd
import json
import csv
import joblib
import numpy as np
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import os
from tensorflow import keras
from keras_bert import get_base_dict, get_model, compile_model, gen_batch_inputs
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ["CUDA_VISIBLE_DEVICES"] = '1,2'

## Load sequence pairs of clinical codes to train BERT model

In [ ]:
import joblib

sentence_pair=joblib.load('sentence_pair_sample.joblib')

In [ ]:
# Build token dictionary
token_dict = get_base_dict()  # A dict that contains some special tokens
for pairs in sentence_pairs:
    for token in pairs[0] + pairs[1]:
        if token not in token_dict:
            token_dict[token] = len(token_dict)
token_list = list(token_dict.keys())  # Used for selecting a random word

## Build & train the model

In [ ]:
# Build & train the model
model = get_model(
    token_num=len(token_dict),
    head_num=4,
    transformer_num=6,
    embed_dim=32,
    feed_forward_dim=20,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
)
compile_model(model)
model.summary()

In [ ]:
def _generator():
    while True:
        yield gen_batch_inputs(
            sentence_pairs_train,
            token_dict,
            token_list,
            seq_len=20,
            mask_rate=0.3,
            swap_sentence_rate=1.0,
        )

model.fit_generator(
    generator=_generator(),
    steps_per_epoch=1000,
    epochs=10,
    validation_data=_generator(),
    validation_steps=100,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    ],
)

## Extract the embedding layer and output the embedding for each clinical code

In [ ]:
embmodel=keras.Model(inputs=model.layers[2].input,outputs=model.layers[2].output[2])

In [ ]:
tokensinput=np.asarray([i for i in range(5,567)]+[0]*18)
tokensinput=tokensinput.reshape((29,20))
tokensemb=embmodel.predict(tokensinput)
np.save('token2emb_bert.npy',tokensemb[1][:562,:])    